# Random Forest
We'll try to load even more features than in notebook 02.

Using all AreaShape besides array values, accuracy 63%.

Just SpacialMoment or just Zernike increased it to 65%.

Hematoxylin granularity increased it to 76%. 

Intensity increased it to 80%. 

Texture contrast increased it to 86%.

All texture incresed it to 88%.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-05-25 17:02:57.257796
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='/home/jrm/Adjeroh/Naved/Process100.class/'
CLASS_DIRS=['0','1','2','3','4','5']
NUM_CLASSES=len(CLASS_DIRS)
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'

In [4]:
# show columns
df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
initial_cols = df.columns
for col in initial_cols:
    print(col)

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [5]:
df = None
GOOD_COLS=['ImageNumber']
GOOD_COLS.extend(['AreaShape_Area','AreaShape_Compactness','AreaShape_ConvexArea'])
#GOOD_COLS.extend(['AreaShape_Eccentricity','AreaShape_EquivalentDiameter','AreaShape_EulerNumber'])
#GOOD_COLS.extend(['AreaShape_Extent','AreaShape_FormFactor','AreaShape_MajorAxisLength'])
#GOOD_COLS.extend(['AreaShape_MaxFeretDiameter','AreaShape_MaximumRadius','AreaShape_MeanRadius'])
#GOOD_COLS.extend(['AreaShape_MinFeretDiameter','AreaShape_MinorAxisLength','AreaShape_Orientation'])
#GOOD_COLS.extend(['AreaShape_Perimeter','AreaShape_Solidity'])
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_NormalizedMoment') ) ] )
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_SpacialMoment') ) ] )  # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_Zernike') ) ] )   # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Granularity') ) ] )   # yes!!!
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Intensity') ) ] )   # yes!!!
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Neighbors') ) ] )   
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('RadialDistribution') ) ] )   
GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_') ) ] )  # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Contrast_Hematoxylin') ) ] )   
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_DifferenceEntropy_Hematoxylin') ) ] ) 
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Entropy_Hematoxylin') ) ] )  # yes
GOOD_COLS

['ImageNumber',
 'AreaShape_Area',
 'AreaShape_Compactness',
 'AreaShape_ConvexArea',
 'Texture_AngularSecondMoment_Eosin_3_00_256',
 'Texture_AngularSecondMoment_Eosin_3_01_256',
 'Texture_AngularSecondMoment_Eosin_3_02_256',
 'Texture_AngularSecondMoment_Eosin_3_03_256',
 'Texture_AngularSecondMoment_Eosin_4_00_256',
 'Texture_AngularSecondMoment_Eosin_4_01_256',
 'Texture_AngularSecondMoment_Eosin_4_02_256',
 'Texture_AngularSecondMoment_Eosin_4_03_256',
 'Texture_AngularSecondMoment_Eosin_5_00_256',
 'Texture_AngularSecondMoment_Eosin_5_01_256',
 'Texture_AngularSecondMoment_Eosin_5_02_256',
 'Texture_AngularSecondMoment_Eosin_5_03_256',
 'Texture_AngularSecondMoment_Eosin_7_00_256',
 'Texture_AngularSecondMoment_Eosin_7_01_256',
 'Texture_AngularSecondMoment_Eosin_7_02_256',
 'Texture_AngularSecondMoment_Eosin_7_03_256',
 'Texture_AngularSecondMoment_Hematoxylin_3_00_256',
 'Texture_AngularSecondMoment_Hematoxylin_3_01_256',
 'Texture_AngularSecondMoment_Hematoxylin_3_02_256',
 'T

In [6]:
def nuc_prefix(col_name):
    return 'Nuc_'+col_name
def load_one_class(class_index):
    class_name=CLASS_DIRS[class_index]+'/'
    object_df = pd.read_csv(BASE_DIR+class_name+NUC_FN)
    print("Load %d nuclei, class %s." % (len(object_df),class_name))
    object_df = object_df[GOOD_COLS]
    object_df.rename(nuc_prefix,axis='columns',inplace=True)
    patch_df = object_df.groupby(['Nuc_ImageNumber']).describe()
    # The random forest classifier cannot handle two-part column names, so convert them to string.    
    patch_df.columns=patch_df.columns.map('_'.join)
    patch_df.fillna(0,inplace=True)
    print("Load %d patches having nuclei." % len(patch_df))
    return patch_df
df = load_one_class(0)

Load 1826 nuclei, class 0/.
Load 100 patches having nuclei.


In [7]:
df

,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_Compactness_count,Nuc_AreaShape_Compactness_mean,...,Nuc_Texture_Variance_Hematoxylin_7_02_256_75%,Nuc_Texture_Variance_Hematoxylin_7_02_256_max,Nuc_Texture_Variance_Hematoxylin_7_03_256_count,Nuc_Texture_Variance_Hematoxylin_7_03_256_mean,Nuc_Texture_Variance_Hematoxylin_7_03_256_std,Nuc_Texture_Variance_Hematoxylin_7_03_256_min,Nuc_Texture_Variance_Hematoxylin_7_03_256_25%,Nuc_Texture_Variance_Hematoxylin_7_03_256_50%,Nuc_Texture_Variance_Hematoxylin_7_03_256_75%,Nuc_Texture_Variance_Hematoxylin_7_03_256_max
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,313.000000,0.000000,313.0,313.00,313.0,313.00,313.0,1.0,1.273323,...,5222.972048,5222.972048,1.0,6012.464844,0.000000,6012.464844,6012.464844,6012.464844,6012.464844,6012.464844
2,3.0,251.000000,79.680612,178.0,208.50,239.0,287.50,336.0,3.0,1.816912,...,1999.395944,3406.000566,3.0,966.200528,940.488383,240.873600,434.888784,628.903968,1328.863992,2028.824015
3,5.0,302.000000,162.260593,182.0,212.00,260.0,271.00,585.0,5.0,1.180711,...,4493.263443,4685.992476,5.0,2998.715245,1595.066722,1254.495177,1531.032359,3104.488227,4282.201786,4821.358674
4,29.0,314.137931,125.210030,177.0,233.00,262.0,340.00,635.0,29.0,1.366962,...,5119.685591,6203.178253,29.0,3819.461802,1203.531211,1659.915624,2926.592344,3869.910770,4825.073284,6126.136134
5,32.0,355.937500,160.779339,180.0,243.00,331.0,375.25,832.0,32.0,1.353498,...,2409.327561,4157.494126,32.0,1810.784518,980.971372,244.232167,1063.633042,1663.717733,2508.162306,4161.067762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2.0,306.500000,47.376154,273.0,289.75,306.5,323.25,340.0,2.0,1.355519,...,2690.939391,3246.464268,2.0,1842.985805,1545.103340,750.432755,1296.709280,1842.985805,2389.262329,2935.538854
97,32.0,394.750000,171.982745,189.0,270.50,352.5,480.75,812.0,32.0,1.347032,...,1351.813378,2375.814449,32.0,1197.667219,612.562480,224.327160,805.420154,1081.060984,1614.764361,2535.747320
98,23.0,318.565217,120.095729,183.0,244.50,285.0,373.00,633.0,23.0,1.392848,...,1348.902434,2291.757707,23.0,1210.659219,663.256910,0.000000,679.835136,1133.466624,1739.855241,2578.575725


In [8]:
print(datetime.datetime.now())
df = None
def load_all_classes():
    X = load_one_class(0)
    size = len(X)
    y = np.zeros(size)
    for i in range(1,NUM_CLASSES):
        Xi = load_one_class(i)
        size = len(Xi)
        yi = np.ones(size) * i
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    return X,y
X,y=load_all_classes()
X

2022-05-25 17:03:28.144698
Load 1826 nuclei, class 0/.
Load 100 patches having nuclei.
Load 2435 nuclei, class 1/.
Load 100 patches having nuclei.
Load 886 nuclei, class 2/.
Load 100 patches having nuclei.
Load 1459 nuclei, class 3/.
Load 100 patches having nuclei.
Load 1713 nuclei, class 4/.
Load 100 patches having nuclei.
Load 3659 nuclei, class 5/.
Load 100 patches having nuclei.


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_Compactness_count,Nuc_AreaShape_Compactness_mean,...,Nuc_Texture_Variance_Hematoxylin_7_02_256_75%,Nuc_Texture_Variance_Hematoxylin_7_02_256_max,Nuc_Texture_Variance_Hematoxylin_7_03_256_count,Nuc_Texture_Variance_Hematoxylin_7_03_256_mean,Nuc_Texture_Variance_Hematoxylin_7_03_256_std,Nuc_Texture_Variance_Hematoxylin_7_03_256_min,Nuc_Texture_Variance_Hematoxylin_7_03_256_25%,Nuc_Texture_Variance_Hematoxylin_7_03_256_50%,Nuc_Texture_Variance_Hematoxylin_7_03_256_75%,Nuc_Texture_Variance_Hematoxylin_7_03_256_max
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,313.000000,0.000000,313.0,313.00,313.0,313.00,313.0,1.0,1.273323,...,5222.972048,5222.972048,1.0,6012.464844,0.000000,6012.464844,6012.464844,6012.464844,6012.464844,6012.464844
2,3.0,251.000000,79.680612,178.0,208.50,239.0,287.50,336.0,3.0,1.816912,...,1999.395944,3406.000566,3.0,966.200528,940.488383,240.873600,434.888784,628.903968,1328.863992,2028.824015
3,5.0,302.000000,162.260593,182.0,212.00,260.0,271.00,585.0,5.0,1.180711,...,4493.263443,4685.992476,5.0,2998.715245,1595.066722,1254.495177,1531.032359,3104.488227,4282.201786,4821.358674
4,29.0,314.137931,125.210030,177.0,233.00,262.0,340.00,635.0,29.0,1.366962,...,5119.685591,6203.178253,29.0,3819.461802,1203.531211,1659.915624,2926.592344,3869.910770,4825.073284,6126.136134
5,32.0,355.937500,160.779339,180.0,243.00,331.0,375.25,832.0,32.0,1.353498,...,2409.327561,4157.494126,32.0,1810.784518,980.971372,244.232167,1063.633042,1663.717733,2508.162306,4161.067762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,47.0,574.255319,290.847448,177.0,305.50,534.0,746.00,1323.0,47.0,1.459147,...,970.315313,2499.384383,47.0,821.277305,424.244956,212.453900,564.352221,780.295996,960.390187,2537.304120
97,5.0,667.600000,270.346629,280.0,513.00,734.0,888.00,923.0,5.0,1.697802,...,1345.543873,2570.088740,5.0,1317.452774,733.072397,701.057175,883.629677,1153.638950,1285.070060,2563.868007
98,50.0,549.580000,293.608798,181.0,347.00,440.0,728.75,1565.0,50.0,1.360029,...,660.479293,1144.578125,50.0,557.211175,274.015959,184.882017,387.009224,491.560962,601.010218,1462.124728


In [9]:
print(datetime.datetime.now())
Xtrain,Xvalid,ytrain,yvalid = train_test_split(X, y.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xvalid',Xvalid.shape,'yvalid',yvalid.shape,'non-zero:',np.count_nonzero(yvalid))

2022-05-25 17:06:22.531439
Xtrain (450, 3352) ytrain (450,) non-zero: 375
Xvalid (150, 3352) yvalid (150,) non-zero: 125


In [10]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model.fit(self.Xtr,self.ytr)
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  # bug fix
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

In [11]:
print('Train on all Features')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
rf1.set_validation(Xvalid,yvalid)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())
print('The impurity-based feature importances.')
top = rf1.important_features()
top.head()

Train on all Features
2022-05-25 17:06:22.548571
2022-05-25 17:06:23.057824
Accuracy: 88.0
Confusion:
[[22  0  1  2  0  0]
 [ 0 29  4  2  0  1]
 [ 1  1 20  2  0  0]
 [ 1  0  0 18  0  0]
 [ 0  0  1  0 26  0]
 [ 0  0  0  0  2 17]]
The impurity-based feature importances.


,0,1
0,0.010397,Nuc_Texture_SumAverage_Eosin_7_03_256_mean
1,0.009235,Nuc_Texture_SumAverage_Eosin_4_01_256_mean
2,0.009033,Nuc_Texture_SumAverage_Eosin_3_01_256_75%
3,0.008744,Nuc_Texture_SumAverage_Eosin_5_00_256_25%
4,0.008107,Nuc_Texture_Contrast_Eosin_3_03_256_75%


GPU was not active:  
2022-05-25 16:35:28.165143  
2022-05-25 16:35:28.524197

GPU probably not active:  
2022-05-25 16:50:42.142147  
2022-05-25 16:50:42.486454

Why isn't pandas using the GPU?  
Why isn't RF using the GPU?